In [33]:
import pandas as pd
import geopandas as gp
import numpy as np
import folium

In [6]:
df = pd.read_csv('C:/Workspace/TNC-Demand-Model/Outputs/Estimation_File.csv', usecols = ['ORIGIN','NUM_TRIPS_POOLED','SHARED_FLAGGER','DESTINATION'])

In [9]:
df['MATCHED_FLAGGER'] = np.where(df['NUM_TRIPS_POOLED'] > 1, 1, 0)

In [12]:
geo = gp.read_file('C:/Workspace/TNC-Demand-Model/Inputs/Census Shapefiles/Illinois/Chicago Tracts/geo_export_558aad9f-98d8-4dd5-a6b1-c1730155d596.shp')

In [13]:
df= df[df['SHARED_FLAGGER'] == 1]

In [14]:
df['TRIPS'] = 1

C:\Users\ramu225\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [16]:
df.columns

Index(['ORIGIN', 'DESTINATION', 'SHARED_FLAGGER', 'NUM_TRIPS_POOLED',
       'MATCHED_FLAGGER', 'TRIPS'],
      dtype='object')

In [17]:
df = df[['ORIGIN','DESTINATION','NUM_TRIPS_POOLED','MATCHED_FLAGGER','TRIPS']].groupby(by = ['ORIGIN','DESTINATION'], as_index = False).sum()

In [18]:
df['PERCENT_MATCHED'] = df['MATCHED_FLAGGER']/df['TRIPS']

In [38]:
geo['ORIGIN'] = pd.to_numeric(geo.geoid10)

Index(['commarea', 'commarea_n', 'countyfp10', 'geoid10', 'name10',
       'namelsad10', 'notes', 'statefp10', 'tractce10', 'geometry'],
      dtype='object')

In [32]:
df_map = geo.merge(df, how = 'left', left_on = 'CENSUS_TRACT', right_on = 'ORIGIN')

In [35]:
chi_map = folium.Map(location= [41.8781, 87.6298], zoom_start=4,tiles='openstreetmap')


In [40]:
custom_scale = (df_map['PERCENT_MATCHED'].quantile((0,0.2,0.4,0.6,0.8,1))).tolist()

folium.Choropleth(
            geo_data= df_map,
            columns=['ORIGIN', 'PERCENT_MATCHED'],  #Here we tell folium to get the county fips and plot new_cases_7days metric for each county
            threshold_scale=custom_scale, #use the custom scale we created for legend
            fill_color='YlOrRd',
            nan_fill_color="White", #Use white color if there is no data available for the county
            fill_opacity=0.7,
            line_opacity=0.2,
            legend_name='Percent of Shared Trips Succesfully Matching', #title of the legend
            highlight=True,
            line_color='black').add_to(chi_map) 

chi_map

ValueError: key_on `'geoid10'` not found in GeoJSON.

In [41]:
df['MATCHED_UTILITY'] = df['PRIVATE_TRAVEL_TIME']*-0.11353 + df['PRIVATE_FARE']*-0.150391 + df['PRIVATE_TAX']*-0.150391
df['UNMATCHED_UTILITY'] = -2.294251 + df['SHARED_TRAVEL_TIME']*-0.11353 + df['SHARED_FARE']*-0.150391 + df['SHARED_TAX']*-0.150391 + df['HHLDS_UNDER_35k_ORIGIN']* 0.010667 + df['HHLDS_UNDER_35k_DESTINATION']* 0.016374 + df['AIRPORT_FLAGGER']*-1.997917